In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler

In [51]:
data_dir = "football-prediction/epl-training.csv"
df = pd.read_csv(data_dir)

In [52]:
df.columns = [
    'Date',               
    'Home Team',          
    'Away Team',
    'Full Time Home Goals',  
    'Full Time Away Goals',  
    'Full Time Result',       
    'Half Time Home Goals',  
    'Half Time Away Goals',  
    'Half Time Result',      
    'Referee',
    'Home Shots',             
    'Away Shots',             
    'Home Shots on Target',   
    'Away Shots on Target',   
    'Home Corners',          
    'Away Corners',           
    'Home Fouls',             
    'Away Fouls',             
    'Home Yellow Cards',     
    'Away Yellow Cards',     
    'Home Red Cards',     
    'Away Red Cards'      
]

In [ ]:
df['Home Goal Conversion Rate'] = df['Full Time Home Goals'] / df['Home Shots on Target'].replace(0, 1)
df['Away Goal Conversion Rate'] = df['Full Time Away Goals'] / df['Away Shots on Target'].replace(0, 1)
df['Goal Conversion Rate Difference'] = df['Home Goal Conversion Rate'] - df['Away Goal Conversion Rate']

df['Home Attacking Intensity'] = df['Home Shots'] + df['Home Corners']
df['Away Attacking Intensity'] = df['Away Shots'] + df['Away Corners']
df['Attacking Intensity Difference'] = df['Home Attacking Intensity'] - df['Away Attacking Intensity']

df['Home Disciplinary Pressure'] = df['Home Fouls'] + df['Home Yellow Cards'] + df['Home Red Cards']
df['Away Disciplinary Pressure'] = df['Away Fouls'] + df['Away Yellow Cards'] + df['Away Red Cards']
df['Disciplinary Pressure Difference'] = df['Home Disciplinary Pressure'] - df['Away Disciplinary Pressure']

df['Match Outcome'] = df['Full Time Result'].map({'H': 1, 'D': 0, 'A': -1})

In [54]:
def compute_features_using_home_team_as_reference(df):
    features_to_differentiate = [
        ('Full Time Home Goals', 'Full Time Away Goals', 'Full Time Goal Difference'),
        ('Half Time Home Goals', 'Half Time Away Goals', 'Half Time Goal Difference'),
        ('Home Shots', 'Away Shots', 'Shot Difference'),
        ('Home Shots on Target', 'Away Shots on Target', 'Shots on Target Difference'),
        ('Home Corners', 'Away Corners', 'Corner Difference'),
        ('Home Fouls', 'Away Fouls', 'Foul Difference'),
        ('Home Yellow Cards', 'Away Yellow Cards', 'Yellow Card Difference'),
        ('Home Red Cards', 'Away Red Cards', 'Red Card Difference')
    ]


    for home_feature, away_feature, new_feature in features_to_differentiate:
        df[new_feature] = df[home_feature] - df[away_feature]

    
    return df

In [55]:
df = compute_features_using_home_team_as_reference(df)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df = df.sort_values(by='Date')
# Each season starts at 08 and ends at 05 of next year - 2000-2001 season will be the 2000 season
def get_season(date):
    if date.month >= 8:  
        return (date.year)
    else:  
        return (date.year - 1)

df['Season'] = df['Date'].apply(get_season)

In [59]:
# calculates the winstreak length at games played with 
def calculate_home_win_streak(df):
    df['Home Win Streak'] = 0
    for team, team_group in df.groupby('Home Team'):
        win_streak = 0
        for idx, row in team_group.iterrows():
            if row['Match Outcome'] == '1': 
                win_streak += 1
            else:
                win_streak = 0 
            df.at[idx, 'Home Win Streak'] = win_streak

    return df
df = calculate_home_win_streak(df)

In [ ]:
def win_streak(df, last_x_games=5):

    def get_recent_performance(team, current_season, current_date, last_x_games):
        team_matches = df[((df['Home Team'] == team) | (df['Away Team'] == team)) & (df['Season'] == current_season)]
        
        team_matches = team_matches[team_matches['Date'] < current_date]
        team_matches = team_matches.sort_values(by='Date')

        recent_matches = team_matches.tail(last_x_games)
        wins = 0
        for _, row in recent_matches.iterrows():
            if row['Home Team'] == team and row['Match Outcome'] == 1:
                wins += 1
            elif row['Home Team'] == team and (row['Match Outcome'] == 0 or row['Match Outcome'] == -1):
                wins = 0
            elif row['Away Team'] == team and row['Match Outcome'] == -1:
                wins += 1
            elif row['Away Team'] == team and (row['Match Outcome'] == 0 or row['Match Outcome'] == 1):
                wins = 0

        return wins

    for idx, row in df.iterrows():
        current_season = row['Season']
        current_date = row['Date']
        home_team = row['Home Team']
        away_team = row['Away Team']
        
        home_performance = get_recent_performance(home_team, current_season, current_date, last_x_games)
        away_performance = get_recent_performance(away_team, current_season, current_date, last_x_games)

        df.loc[idx, 'Recent Performance Home Team'] = home_performance
        df.loc[idx, 'Recent Performance Away Team'] = away_performance
    return df

df = win_streak(df, last_x_games=5)

In [60]:
df.sort_values(by='Date', inplace=True)

In [93]:
df.columns

Index(['Date', 'Home Team', 'Away Team', 'Full Time Home Goals',
       'Full Time Away Goals', 'Full Time Result', 'Half Time Home Goals',
       'Half Time Away Goals', 'Half Time Result', 'Referee', 'Home Shots',
       'Away Shots', 'Home Shots on Target', 'Away Shots on Target',
       'Home Corners', 'Away Corners', 'Home Fouls', 'Away Fouls',
       'Home Yellow Cards', 'Away Yellow Cards', 'Home Red Cards',
       'Away Red Cards', 'Home Goal Conversion Rate',
       'Away Goal Conversion Rate', 'Goal Conversion Rate Difference',
       'Home Attacking Intensity', 'Away Attacking Intensity',
       'Attacking Intensity Difference', 'Home Disciplinary Pressure',
       'Away Disciplinary Pressure', 'Disciplinary Pressure Difference',
       'Match Outcome', 'Full Time Goal Difference',
       'Half Time Goal Difference', 'Shot Difference',
       'Shots on Target Difference', 'Corner Difference', 'Foul Difference',
       'Yellow Card Difference', 'Red Card Difference', 'Season'

In [85]:
df['Delete Row'] = 0

def avg_goals_scored_conceded_last_x_games(df, max_x=5):
    
    def calculate_avgs(team, current_season, match_date, x):
        relevant_matches = df[
            ((df['Home Team'] == team) | (df['Away Team'] == team)) & 
            ((df['Season'] == current_season) | (df['Season'] == current_season - 1))
        ].sort_values(by='Date')

        past_matches = relevant_matches[relevant_matches['Date'] < match_date]
        recent_matches = past_matches.tail(x)
        goals_scored, goals_conceded = 0, 0

        for _, row in recent_matches.iterrows():
            if row['Home Team'] == team:
                goals_scored += row['Full Time Home Goals']
                goals_conceded += row['Full Time Away Goals']
            elif row['Away Team'] == team:
                goals_scored += row['Full Time Away Goals']
                goals_conceded += row['Full Time Home Goals']

        num_matches = len(recent_matches)
        avg_scored = goals_scored / num_matches if num_matches > 0 else None
        avg_conceded = goals_conceded / num_matches if num_matches > 0 else None

        return avg_scored, avg_conceded

    for x in range(1, max_x + 1):
        df[f'Avg Goals Scored Home Last {x} Games'] = 0.0
        df[f'Avg Goals Conceded Home Last {x} Games'] = 0.0
        df[f'Avg Goals Scored Away Last {x} Games'] = 0.0
        df[f'Avg Goals Conceded Away Last {x} Games'] = 0.0

    running_stats = {}

    for idx, row in df.iterrows():
        for team_type in ['Home Team', 'Away Team']:
            team = row[team_type]
            current_season = row['Season']
            match_date = row['Date']

            if current_season not in running_stats:
                running_stats[current_season] = {}

            if team not in running_stats[current_season]:
                running_stats[current_season][team] = {
                    'scored': [],
                    'conceded': []
                }

            team_stats = running_stats[current_season][team]

            for x in range(1, max_x + 1):
                if len(team_stats['scored']) < x:
                    df.at[idx, f'Avg Goals Scored {team_type.split()[0]} Last {x} Games'] = None
                    df.at[idx, f'Avg Goals Conceded {team_type.split()[0]} Last {x} Games'] = None
                    df.at[idx, 'Delete Row'] = 1
                else:
                    avg_scored, avg_conceded = calculate_avgs(team, current_season, match_date, x)
                    df.at[idx, f'Avg Goals Scored {team_type.split()[0]} Last {x} Games'] = avg_scored
                    df.at[idx, f'Avg Goals Conceded {team_type.split()[0]} Last {x} Games'] = avg_conceded

            team_stats['scored'].append(row['Full Time Home Goals'] if team_type == 'Home Team' else row['Full Time Away Goals'])
            team_stats['conceded'].append(row['Full Time Away Goals'] if team_type == 'Home Team' else row['Full Time Home Goals'])

    return df


In [87]:
df = avg_goals_scored_conceded_last_x_games(df)

In [112]:
def avg_stats_last_x_games(df, max_x=5):
    
    def calculate_running_avg(stats_list, x):
        if len(stats_list) < x:
            return None
        return sum(stats_list[-x:]) / x

    df = df.sort_values(by='Date')

    metrics = ['Goals Scored', 'Goals Conceded', 'xG', 'Disciplinary Pressure']
    for metric in metrics:
        for x in range(1, max_x + 1):
            df[f'Avg {metric} Home Last {x} Games'] = None
            df[f'Avg {metric} Away Last {x} Games'] = None

    running_stats = {}

    for idx, row in df.iterrows():
        for team_type in ['Home Team', 'Away Team']:
            team = row[team_type]
            stat_type = team_type.split()[0]  # Home or Away

            if team not in running_stats:
                running_stats[team] = {
                    'Goals Scored': [],
                    'Goals Conceded': [],
                    'xG': [],
                    'Disciplinary Pressure': [],
                }

            team_stats = running_stats[team]

            for x in range(1, max_x + 1):
                for metric in metrics:
                    avg_stat = calculate_running_avg(team_stats[metric], x)
                    df.at[idx, f'Avg {metric} {stat_type} Last {x} Games'] = avg_stat

            team_stats['Goals Scored'].append(row['Full Time Home Goals'] if stat_type == 'Home' else row['Full Time Away Goals'])
            team_stats['Goals Conceded'].append(row['Full Time Away Goals'] if stat_type == 'Home' else row['Full Time Home Goals'])
            team_stats['xG'].append(row['xG Home'] if stat_type == 'Home' else row['xG Away'])
            team_stats['Disciplinary Pressure'].append(row['Home Disciplinary Pressure'] if stat_type == 'Home' else row['Away Disciplinary Pressure'])

    return df



In [113]:
df = avg_stats_last_x_games(df)

In [114]:
df

,Date,Home Team,Away Team,Full Time Home Goals,Full Time Away Goals,Full Time Result,Half Time Home Goals,Half Time Away Goals,Half Time Result,Referee,...,Avg Disciplinary Pressure Home Last 1 Games,Avg Disciplinary Pressure Away Last 1 Games,Avg Disciplinary Pressure Home Last 2 Games,Avg Disciplinary Pressure Away Last 2 Games,Avg Disciplinary Pressure Home Last 3 Games,Avg Disciplinary Pressure Away Last 3 Games,Avg Disciplinary Pressure Home Last 4 Games,Avg Disciplinary Pressure Away Last 4 Games,Avg Disciplinary Pressure Home Last 5 Games,Avg Disciplinary Pressure Away Last 5 Games
0,2000-08-19,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,Rob Harris,...,None,None,None,None,None,None,None,None,None,None
1,2000-08-19,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,Graham Barber,...,None,None,None,None,None,None,None,None,None,None
2,2000-08-19,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,Barry Knight,...,None,None,None,None,None,None,None,None,None,None
3,2000-08-19,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,Andy D'Urso,...,None,None,None,None,None,None,None,None,None,None
4,2000-08-19,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,Dermot Gallagher,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9211,2024-05-19,Arsenal,Everton,2.0,1.0,H,1.0,1.0,D,M Oliver,...,9.0,4.0,8.5,7.0,8.0,7.0,7.5,6.25,6.8,6.8
9213,2024-05-19,Brighton,Man United,0.0,2.0,A,0.0,0.0,D,C Pawson,...,6.0,12.0,7.0,8.0,7.666667,6.666667,6.75,6.75,6.6,7.2
9212,2024-05-19,Brentford,Newcastle,2.0,4.0,A,0.0,3.0,A,S Hooper,...,10.0,12.0,9.0,11.5,8.666667,12.0,9.25,10.0,8.6,9.0
9219,2024-05-19,Man City,West Ham,3.0,1.0,H,2.0,1.0,H,J Brooks,...,7.0,8.0,7.0,8.5,6.666667,7.333333,6.25,6.0,5.8,6.6


In [ ]:
df["Avg Disciplinary Pressure Home Last 1 Games"].value_counts()

Avg Disciplinary Pressure Home Last 1 Games
0.0     7086
12.0     197
11.0     166
13.0     159
10.0     158
14.0     154
9.0      151
15.0     133
8.0      116
16.0      99
17.0      85
7.0       83
18.0      71
6.0       63
19.0      55
20.0      46
5.0       37
4.0       21
22.0      18
21.0      18
23.0      14
3.0       10
26.0      10
24.0       8
2.0        6
25.0       5
27.0       5
28.0       3
32.0       1
Name: count, dtype: int64

In [63]:
def proxy_xg(df):
    df['xG Home'] = df['Home Goal Conversion Rate'] * df['Home Attacking Intensity']
    df['xG Away'] = df['Away Goal Conversion Rate'] * df['Away Attacking Intensity']
    df['xG Difference'] = df['xG Home'] - df['xG Away']
    return df
df = proxy_xg(df)

In [72]:
def calculate_season_points(df):
    df['Home Team Points'] = 0
    df['Away Team Points'] = 0

    for idx, row in df.iterrows():
        if row['Match Outcome'] == 1:  
            df.at[idx, 'Home Team Points'] = 3
            df.at[idx, 'Away Team Points'] = 0
        elif row['Match Outcome'] == 0: 
            df.at[idx, 'Home Team Points'] = 1
            df.at[idx, 'Away Team Points'] = 1
        elif row['Match Outcome'] == -1:  
            df.at[idx, 'Home Team Points'] = 0
            df.at[idx, 'Away Team Points'] = 3

    df['Home Total Points'] = (df.groupby(['Home Team', 'Season'])['Home Team Points'].cumsum())
    df['Away Total Points'] = (df.groupby(['Away Team', 'Season'])['Away Team Points'].cumsum())
    df['Point Difference'] = df['Home Total Points'] - df['Away Total Points']
    df.drop(columns=['Home Team Points', 'Away Team Points'], inplace=True)

    return df
df = calculate_season_points(df)

In [74]:
df.columns

Index(['Date', 'Home Team', 'Away Team', 'Full Time Home Goals',
       'Full Time Away Goals', 'Full Time Result', 'Half Time Home Goals',
       'Half Time Away Goals', 'Half Time Result', 'Referee', 'Home Shots',
       'Away Shots', 'Home Shots on Target', 'Away Shots on Target',
       'Home Corners', 'Away Corners', 'Home Fouls', 'Away Fouls',
       'Home Yellow Cards', 'Away Yellow Cards', 'Home Red Cards',
       'Away Red Cards', 'Home Goal Conversion Rate',
       'Away Goal Conversion Rate', 'Goal Conversion Rate Difference',
       'Home Attacking Intensity', 'Away Attacking Intensity',
       'Attacking Intensity Difference', 'Home Disciplinary Pressure',
       'Away Disciplinary Pressure', 'Disciplinary Pressure Difference',
       'Match Outcome', 'Full Time Goal Difference',
       'Half Time Goal Difference', 'Shot Difference',
       'Shots on Target Difference', 'Corner Difference', 'Foul Difference',
       'Yellow Card Difference', 'Red Card Difference', 'Season'

In [75]:
df['Difference Average Goals Scored Last X Games'] = df['Avg Goals Scored Home Last X Games'] - df['Avg Goals Scored Away Last X Games']
df['Difference Average Goals Conceded Last X Games'] = df['Avg Goals Conceded Home Last X Games'] - df['Avg Goals Conceded Away Last X Games']

In [76]:
def clean_up_columns(df):
    columns_to_keep = [
        'Home Team', 'Away Team', 'Home Win Streak', 'Difference Average Goals Scored Last X Games', 'Difference Average Goals Conceded Last X Games',
        'Point Difference', 'xG Difference', 'Recent Performance Home Team', 'Recent Performance Away Team', 'Disciplinary Pressure Difference', 'Match Outcome'
    ]
    
    df = df[columns_to_keep]
    return df

In [79]:
df = df.drop(df.index[-1])

In [ ]:
# one hot encoding - use for linear models
df = pd.get_dummies(df, columns=['Home Team', 'Away Team'], drop_first=True)

In [ ]:
# label encoding - use for boosting models
all_teams = pd.concat([df['Home Team'], df['Away Team']]).unique()
le = LabelEncoder()
le.fit(all_teams)
df['Home Team'] = le.transform(df['Home Team'])
df['Away Team'] = le.transform(df['Away Team'])

In [ ]:
"""
If you plan to use tree-based models like XGBoost or Random Forest, normalization and standardization are generally not required since these models are not sensitive to feature scaling. 
However, you still need to handle the categorical features properly (one-hot or label encoding).

Normalize or Standardize the continuous features (Avg Goals, Point Difference, xG Difference) if you're using algorithms sensitive to feature scale (like KNN, SVM, or Neural Networks).
Regularization: Choose based on the model (linear models or neural networks).
DO NOT USE STANDARIZATION AND NORMALIZATION AT THE SAME TIME
LINEAR REGRESIION (OUR BENCHMARK) -->> STANDARIZATION
"""

In [ ]:
"""
Normalization (scaling features to have a range between 0 and 1) is typically used when you have features that vary greatly in scale (e.g., Avg Goals Scored might range from 0 to 5, whereas Home Win Streak might range from 0 to 10).
Models like KNN, neural networks, and SVMs generally perform better when the data is normalized.
For features like xG Difference, Avg Goals Scored, or Point Difference, normalizing them can help ensure that they contribute equally to the model's performance.
"""
scaler = MinMaxScaler()
df[['Difference Average Goals Scored Last X Games', 'Difference Average Goals Conceded Last X Games', 'Point Difference', 'xG Difference']] = scaler.fit_transform(df[['Difference Average Goals Scored Last X Games', 'Difference Average Goals Conceded Last X Games', 'Point Difference', 'xG Difference']])

In [ ]:
"""
Standardization scales the data to have a mean of 0 and a standard deviation of 1. This is useful when features have different ranges and you want to treat them equally.
It is essential for models like logistic regression or SVM when you want to avoid bias towards larger values in the features.
"""
scaler = StandardScaler()
df[['Difference Average Goals Scored Last X Games', 'Difference Average Goals Conceded Last X Games', 'Point Difference', 'xG Difference']] = scaler.fit_transform(df[['Difference Average Goals Scored Last X Games', 'Difference Average Goals Conceded Last X Games', 'Point Difference', 'xG Difference']])